GPAの2値分類

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [19]:
df=pd.read_csv('../data/Classification-of-students.csv')

カテゴリカルデータtrain_cum.csv validation_num.csvの作成

In [20]:
df.head()

,StudentID,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA,GradeClass
0,1001,17,1,0,2,19.833723,7,1,2,0,0,1,0,2.929196,2.0
1,1002,18,0,0,1,15.408756,0,0,1,0,0,0,0,3.042915,1.0
2,1003,15,0,2,3,4.210570,26,0,2,0,0,0,0,0.112602,4.0
3,1004,17,1,0,3,10.028829,14,0,3,1,0,0,0,2.054218,3.0
4,1005,17,1,0,2,4.672495,17,1,3,0,0,0,0,1.288061,4.0


In [21]:
df['GradeClass']=df['GradeClass'].astype(int)
df['StudyTimeWeekly']=df['StudyTimeWeekly'].astype(int)
df=df.drop('StudentID',axis=1)
df=df.drop('Age',axis=1)
df=df.drop('Absences',axis=1)
df=df.drop('GradeClass',axis=1)

In [22]:
df.loc[df['GPA']<=2,'target']=0
df.loc[(df['GPA']>2)&(df['GPA']<=4),'target']=1

In [23]:
df['target']=df['target'].astype(int)
df=df.drop('GPA',axis=1)

In [24]:
df.head()

,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,target
0,1,0,2,19,1,2,0,0,1,0,1
1,0,0,1,15,0,1,0,0,0,0,1
2,0,2,3,4,0,2,0,0,0,0,0
3,1,0,3,10,0,3,1,0,0,0,1
4,1,0,2,4,1,3,0,0,0,0,0


In [25]:
df.shape

(2392, 11)

In [26]:
df['target'].value_counts()

target
0    1274
1    1118
Name: count, dtype: int64

In [27]:
train_num,validation_num=df[:int(len(df)*0.8)],df[int(len(df)*0.8):].reset_index(drop=True)

In [28]:
train_num['target'].value_counts()

target
0    1020
1     893
Name: count, dtype: int64

In [29]:
train_num.head()

,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,target
0,1,0,2,19,1,2,0,0,1,0,1
1,0,0,1,15,0,1,0,0,0,0,1
2,0,2,3,4,0,2,0,0,0,0,0
3,1,0,3,10,0,3,1,0,0,0,1
4,1,0,2,4,1,3,0,0,0,0,0


In [30]:
validation_num.head()

,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,target
0,1,0,1,12,0,3,0,1,0,0,0
1,1,0,3,8,0,1,1,0,0,0,1
2,1,1,1,1,0,1,1,0,1,0,1
3,1,2,2,9,1,4,0,1,0,0,1
4,0,2,2,14,1,0,0,0,0,0,1


In [31]:
train_num.to_csv('../data/2class_classification_data/train_num.csv',index=False)
validation_num.to_csv('../data/2class_classification_data/validation_num.csv',index=False)

カテゴリカルデータからテキストデータに変換

In [32]:
Gender_mapping={0:'男性は',1:'女性は'}
Ethnicity_mapping={0: '白人で、',1: '黒人で、',2:'アジア人で、',3: '他の人種で、'}
ParentalEducation_mapping={0: '親は中学卒業で、',1: '親は高校卒業で、',2: '親は短期大学もしくは専門学校卒業で、',3: '親は大学卒業で、',4: '親は大学院卒業で、'}
Tutoring_mapping={0:'本人は個別指導経験ありで、',1:'本人は個別指導経験なしで、'}
ParentalSupport_mapping={0: '親の援助は全くなく、',1: '親の援助を少し受けていて、',2: '親の援助は普通で、',3:'親の援助は大きく、',4: '親の援助は非常に大きく、'}
Extracurricular_mapping={0:'塾には通っておらず、',1:'塾に通っていて、'}
Sports_mapping={0:'スポーツはしていなくて、',1:'スポーツをしていて、'}
Music_mapping={0:'音楽には無関心で、',1:'音楽に精通していて、'}
Volunteering_mapping={0:'ボランティア活動はしていない。',1:'ボランティア活動をしている。'}


In [33]:
df['Gender']=df['Gender'].map(Gender_mapping)
df['Ethnicity']=df['Ethnicity'].map(Ethnicity_mapping)
df['ParentalEducation']=df['ParentalEducation'].map(ParentalEducation_mapping)
df['Tutoring']=df['Tutoring'].map(Tutoring_mapping)
df['ParentalSupport']=df['ParentalSupport'].map(ParentalSupport_mapping)
df['Extracurricular']=df['Extracurricular'].map(Extracurricular_mapping)
df['Sports']=df['Sports'].map(Sports_mapping)
df['Music']=df['Music'].map(Music_mapping)
df['Volunteering']=df['Volunteering'].map(Volunteering_mapping)

In [34]:
text_data = df.apply(lambda row: f"{row['Gender']}{row['Ethnicity']}"+"1週間の勉強時間は"+f"{row['StudyTimeWeekly']}"+"時間で、"+f"{row['ParentalEducation']}{row['Tutoring']}{row['ParentalSupport']}{row['Extracurricular']}{row['Sports']}{row['Music']}{row['Volunteering']}", axis=1)

In [35]:
text_df = pd.DataFrame({'label':df['target'],'sentence': text_data})

In [36]:
train,validation=text_df[:int(len(text_data)*0.8)],text_df[int(len(text_data)*0.8):].reset_index(drop=True)

In [37]:
train.head()

,label,sentence
0,1,女性は白人で、1週間の勉強時間は19時間で、親は短期大学もしくは専門学校卒業で、本人は個別指...
1,1,男性は白人で、1週間の勉強時間は15時間で、親は高校卒業で、本人は個別指導経験ありで、親の援...
2,0,男性はアジア人で、1週間の勉強時間は4時間で、親は大学卒業で、本人は個別指導経験ありで、親の...
3,1,女性は白人で、1週間の勉強時間は10時間で、親は大学卒業で、本人は個別指導経験ありで、親の援...
4,0,女性は白人で、1週間の勉強時間は4時間で、親は短期大学もしくは専門学校卒業で、本人は個別指導...


In [38]:
validation.head()

,label,sentence
0,0,女性は白人で、1週間の勉強時間は12時間で、親は高校卒業で、本人は個別指導経験ありで、親の援...
1,1,女性は白人で、1週間の勉強時間は8時間で、親は大学卒業で、本人は個別指導経験ありで、親の援助...
2,1,女性は黒人で、1週間の勉強時間は1時間で、親は高校卒業で、本人は個別指導経験ありで、親の援助...
3,1,女性はアジア人で、1週間の勉強時間は9時間で、親は短期大学もしくは専門学校卒業で、本人は個別...
4,1,男性はアジア人で、1週間の勉強時間は14時間で、親は短期大学もしくは専門学校卒業で、本人は個...


In [39]:
train.to_csv('../data/2class_classification_data/train.csv',index=False,encoding='utf-8-sig')
validation.to_csv('../data/2class_classification_data/validation.csv',index=False,encoding='utf-8-sig')